In [34]:
import tkinter as tk

# Examples

## Temperature Converter

In [35]:
def fahrenheit_to_celsius():
    """Convert the value for Fahrenheit to Celsius and insert the
    result into lbl_result.
    """
    fahrenheit = ent_temperature.get()
    celsius = (5 / 9) * (float(fahrenheit) - 32)
    lbl_result["text"] = f"{round(celsius, 2)} \N{DEGREE CELSIUS}"

# Set up the window
window = tk.Tk()
window.title("Temperature Converter")
window.resizable(width=False, height=False)

# Create the Fahrenheit entry frame with an Entry
# widget and label in it
frm_entry = tk.Frame(master=window)
ent_temperature = tk.Entry(master=frm_entry, width=10)
lbl_temp = tk.Label(master=frm_entry, text="\N{DEGREE FAHRENHEIT}")

# Layout the temperature Entry and Label in frm_entry
# using the .grid() geometry manager
ent_temperature.grid(row=0, column=0, sticky="e")
lbl_temp.grid(row=0, column=1, sticky="w")

# Create the conversion Button and result display Label
btn_convert = tk.Button(
    master=window,
    text="\N{RIGHTWARDS BLACK ARROW}",
    command=fahrenheit_to_celsius
)
lbl_result = tk.Label(master=window, text="\N{DEGREE CELSIUS}")

# Set up the layout using the .grid() geometry manager
frm_entry.grid(row=0, column=0, padx=10)
btn_convert.grid(row=0, column=1, pady=10)
lbl_result.grid(row=0, column=2, padx=10)

# Run the application
window.mainloop()

## Text Editor

In [36]:
from tkinter.filedialog import askopenfilename, asksaveasfilename

def open_file():
    """Open a file for editing."""
    filepath = askopenfilename(
        filetypes=[("Text Files", "*.txt"), ("All Files", "*.*")]
    )
    if not filepath:
        return
    txt_edit.delete("1.0", tk.END)
    with open(filepath, mode="r", encoding="utf-8") as input_file:
        text = input_file.read()
        txt_edit.insert(tk.END, text)
    window.title(f"Simple Text Editor - {filepath}")

def save_file():
    """Save the current file as a new file."""
    filepath = asksaveasfilename(
        defaultextension=".txt",
        filetypes=[("Text Files", "*.txt"), ("All Files", "*.*")],
    )
    if not filepath:
        return
    with open(filepath, mode="w", encoding="utf-8") as output_file:
        text = txt_edit.get("1.0", tk.END)
        output_file.write(text)
    window.title(f"Simple Text Editor - {filepath}")

window = tk.Tk()
window.title("Simple Text Editor")

window.rowconfigure(0, minsize=800, weight=1)
window.columnconfigure(1, minsize=800, weight=1)

txt_edit = tk.Text(window)
frm_buttons = tk.Frame(window, relief=tk.RAISED, bd=2)
btn_open = tk.Button(frm_buttons, text="Open", command=open_file)
btn_save = tk.Button(frm_buttons, text="Save As...", command=save_file)

btn_open.grid(row=0, column=0, sticky="ew", padx=5, pady=5)
btn_save.grid(row=1, column=0, sticky="ew", padx=5)

frm_buttons.grid(row=0, column=0, sticky="ns")
txt_edit.grid(row=0, column=1, sticky="nsew")

window.mainloop()

# Backend Code Scratch

In [37]:
import wikipedia

search = "worthington ohio schools"

search_results = wikipedia.search(query=search, results=5)

# Print the top 5 results (titles)
print(f'Top 5 Wikipedia Results for "{search}":')
for result in search_results:
  print(result)

Top 5 Wikipedia Results for "worthington ohio schools":
Worthington, Ohio
Thomas Worthington High School
Worthington Christian High School
Worthington Kilbourne High School
Worthington City School District


# DuckDuckGo and Reddit

In [42]:
from duckduckgo_search import DDGS

results = DDGS().text("best bluetooth headphones site:reddit.com", max_results=10)
display(results)

[{'title': 'What are the best wireless, noise cancelling over-the-ear headphones ...',
  'href': 'https://www.reddit.com/r/HeadphoneAdvice/comments/syc70l/what_are_the_best_wireless_noise_cancelling/',
  'body': 'Here you can ask which headphone is better, which amp you should buy, which gear best fit your budget and anything related to headphone comparisons and purchasing. Members Online Headphone novice looking for closed back headphone recommendations - same budget scale as Sennheiser 600/58x'},
 {'title': 'What are the best sounding wireless headphones? (No Budget)',
  'href': 'https://www.reddit.com/r/HeadphoneAdvice/comments/pf4s5i/what_are_the_best_sounding_wireless_headphones_no/',
  'body': "The Sennheiser PXC 550-II sounds most neutral, and the Sennheiser Momentum 3 sounded best among the V-shaped tuning (boosted bass and treble, scooped-out mids) that's so common with consumer wireless headphones these days. You also could consider IEMs, which considerably deepen the rabbit 

In [69]:
import praw
import json
# Read-only instance
with open("../secrets.json") as f:
    secrets = json.load(f)

reddit = praw.Reddit(
    client_id=secrets.get("client_id"),
    client_secret=secrets.get("client_secret"),
    user_agent=secrets.get("user_agent"),
)

url = "https://www.reddit.com/r/headphones/comments/z5mcei/im_not_an_audiophile_but_these_are_the_best/"
submission = reddit.submission(url=url)
submission.comments.replace_more(limit=None)
for comment in submission.comments.list():
    print(f"(upvotes-{comment.score}) {comment.author}: {comment.body}")

(upvotes-5) chucuyo: Do you happen to use glasses? Very much interested in finding the most comfortable option possible for BT cans. Looking at Sony WH-1000XM4s currently  

Beyer DT770s are a bit tight for me after a while of use, for reference
(upvotes-6) Raytheon-6: The BT headphones I've tried in the past were Bose 700, Sony-1000XM3 and XM4's, and the Momentum 3s. All of them are "flagship" quality BT headphones, but in terms of comfort + sound quality, the 2 most important factors I am looking for, these are my endgame wireless headphones. The sound quality is comparable to the Momentum 3s, but in my opinion, these headphones are close to the Sony and Bose in comfort but blows the Sony and Bose out of the water in sound quality. The bass sounds richer, and the soundstage sounds more expansive. I bought these for $225 renewed off of Amazon, and I have absolutely no regrets.
(upvotes-3) redhotpisser: What model are they?
(upvotes-2) Butch1972: Those look so comfortable.
(upvotes-2) 